In [ ]:
%cd ..

# Load Data 

## Load Raw

In [ ]:
# raw Data

import os 
import pandas as pd

# read the raw data here.
RecName = 'P'
folder = 'data/Simulations/'
raw_file_path =  os.path.join(folder, RecName) + '.csv'
print(raw_file_path)

# load all records
df = pd.read_csv(raw_file_path)
df

## Select Columns

In [ ]:
focal_ids = ['PID']
columns = ['age', 'basicInfo']

df = df[focal_ids + columns]
df

## Basic Filtering

Cohort () & Time range ()

In [ ]:
# nothing

## DF Rec Raw


with the name of `df_RecRaw`.

In [ ]:
df_RecRaw = df

# Fields in Records

In [ ]:
df = df_RecRaw.copy()

## Field: ID

In [ ]:
RecName = 'P'
RID = RecName + 'ID'
print(RID)
df.head()

## Field: age

In [ ]:
df['age']

In [ ]:
# df['age'].isna().sum()

## Field: basicInfo

In [ ]:
df['basicInfo']

## Pin Down

In [ ]:
focal_ids = [RID]
fields_cols = ['age', 'basicInfo']

df_proc = df[focal_ids + fields_cols]
df_proc

# Save and Load

## Save

In [ ]:
from recfldgrn.datapoint import convert_PID_to_PIDgroup
from recfldgrn.datapoint import RANGE_SIZE, write_df_to_folders, load_df_data_from_folder
RANGE_SIZE = 10000; IDNAME = 'PID'

In [ ]:
df_proc

In [ ]:
# the folder to save the Record.
###############
RecName = RecName
df = df_proc
###############

rec_folder = 'Data/ProcData/RecFolder'
data_folder = os.path.join(rec_folder, RecName)
print(data_folder, '<----- fullrec_folder to save the records')

if not os.path.exists(data_folder): 
    os.makedirs(data_folder)

write_df_to_folders(RecName, data_folder, df, IDNAME)

## Load

In [ ]:
from recfldgrn.datapoint import load_df_data_with_RecName
df = load_df_data_with_RecName(rec_folder, RecName)
print(df.shape)
df.head()

## Load One Sample

In [ ]:
from recfldgrn.datapoint import DataPoint
####################
PIDValue = 'P5'  
######################
Pat = DataPoint(IDNAME, PIDValue, rec_folder, RANGE_SIZE)
Pat

In [ ]:
RecName

In [ ]:
Pat.get_df_rec(RecName)